In [0]:
# Download groundtruth file for evaluatation
!wget https://www.robots.ox.ac.uk/~vgg/data/oxbuildings/gt_files_170407.tgz

!mkdir groundtruth
!tar -xf gt_files_170407.tgz -C groundtruth

--2020-01-24 13:13:27--  https://www.robots.ox.ac.uk/~vgg/data/oxbuildings/gt_files_170407.tgz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8174 (8.0K) [application/x-gzip]
Saving to: ‘gt_files_170407.tgz.1’

gt_files_170407.tgz 100%[===================>]   7.98K  --.-KB/s    in 0s      

2020-01-24 13:13:29 (67.4 MB/s) - ‘gt_files_170407.tgz.1’ saved [8174/8174]

mkdir: cannot create directory ‘groundtruth’: File exists


In [0]:
import gdown

# data vlad
url = 'https://drive.google.com/uc?id=1NBOBoZr0bEiMYXbNOyGZ70m2sSNUBo54'
gdown.download(url, 'vlad_data', quiet=False)

# queries vlad
url = 'https://drive.google.com/uc?id=1WTZVARpvZ1Ku3lawsz3V8DLPwfne0ypN'
gdown.download(url, 'vlad_queries', quiet=False)

# data name
url = 'https://drive.google.com/uc?id=1k95RCX9ZtJNCC7m35dokAorjQHajufqF'
gdown.download(url, 'names_data', quiet=False)

# queries name
url = 'https://drive.google.com/uc?id=1k95RCX9ZtJNCC7m35dokAorjQHajufqF'
gdown.download(url, 'names_queries', quiet=False)



Downloading...
From: https://drive.google.com/uc?id=1NBOBoZr0bEiMYXbNOyGZ70m2sSNUBo54
To: /content/vlad_data
663MB [00:02, 231MB/s]
Downloading...
From: https://drive.google.com/uc?id=1WTZVARpvZ1Ku3lawsz3V8DLPwfne0ypN
To: /content/vlad_queries
7.21MB [00:00, 104MB/s]
Downloading...
From: https://drive.google.com/uc?id=1k95RCX9ZtJNCC7m35dokAorjQHajufqF
To: /content/names_data
100%|██████████| 125k/125k [00:00<00:00, 65.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1k95RCX9ZtJNCC7m35dokAorjQHajufqF
To: /content/names_queries
100%|██████████| 125k/125k [00:00<00:00, 74.1MB/s]


'names_queries'

In [0]:
import os
import sys
import h5py
import pickle

import numpy as np

In [0]:
with h5py.File('vlad_data', 'r') as hf:
    vlad_data = hf['vlad'][:]
with h5py.File('vlad_queries', 'r') as hf:
    vlad_queries = hf['descriptors'][:]
with open('names_data', 'rb') as f:
    names_data = pickle.load(f)
with open('names_queries', 'rb') as f:
    names_queries = pickle.load(f)

names_data = np.array(names_data)
names_queries = np.array(names_queries)

In [0]:
# signed square root
def ssr(x):
    return np.sign(x) * np.sqrt(np.abs(x))

# l2 norm
def norm(x, eps=1e-12):
    norms = np.sqrt(np.sum(x**2))

    norms = max(norms, eps)
    return x / norms

# intra norm
def innorm(x, eps=1e-12):
    norms = np.sqrt(np.sum(x**2, axis=1))
    norms[norms < eps] = eps
    return x / norms.reshape(-1, 1)

In [0]:
methods = [
           "ssr1", # ssr before pushing to Vlad data
           "norm1", # norm before pushing to vlad
           "innorm1", # innorm before push to vlad

           "ssr2", # ssr vlad
           "norm2", # norm vlad
           "innorm2", # innorm vlad

           "ssr3", # ssr queries
           "norm3", # norm queries
           "innorm3", # innorm queries
           ]
           

In [0]:
def gen_cond(cond):
    cond_dict = {}
    cond_name_list = []
    for m in methods:
        cond_dict[m] = cond % 2
        if cond_dict[m]:
            cond_name_list.append(m)
        cond = cond >> 1
    cond_name = "_".join(cond_name_list)
    return cond_dict, cond_name

def gen_data(V, cond_dict):
    V2 = []
    for v in V:
        if cond_dict["ssr1"]:
            v = ssr(v)
        if cond_dict["norm1"]:
            v = norm(v)
        if cond_dict["innorm1"]:
            v = innorm(v)
        v = v.flatten()
        V2.append(v)
    V2 = np.array(V2)
    if cond_dict["ssr2"]:
        V2 = ssr(V2)
    if cond_dict["norm2"]:
        V2 = norm(V2)
    if cond_dict["innorm2"]:
        V2 = innorm(V2)
    return V2

def gen_query(v, cond_dict):
    if cond_dict["ssr3"]:
        v = ssr(v) 
    if cond_dict["norm3"]:
        v = norm(v)
    if cond_dict["innorm3"]:
        v = innorm(v)
    v = v.flatten()
    return v

def retrieve(V, v, names_data):
    # euclid rank
    euclid_rank = ((V - v)**2).sum(axis=1).flatten()
    # cosine rank
    cosine_rank = - np.dot(v, V.T)

    euclid_indices = euclid_rank.argsort()
    cosine_indices = cosine_rank.argsort()
    return names_data[euclid_indices], names_data[cosine_indices]

Name 
euclid [0.043364   0.04356745 0.04372815 0.04431068]
cosine [0.29498697 0.32267952 0.33958437 0.34862485]

Name ssr1
euclid [0.45947857 0.49178846 0.50746618 0.51420787]
cosine [0.43924813 0.47062609 0.48676879 0.49419208]

Name norm1
euclid [0.4661974  0.49737189 0.51466651 0.52214819]
cosine [0.46578008 0.49712903 0.51436797 0.52176297]

Name ssr1_norm1
euclid [0.47989567 0.5131107  0.53000482 0.53673052]
cosine [0.48027986 0.5134136  0.53000752 0.53689597]

Name innorm1
euclid [0.49743563 0.53003162 0.54682595 0.55346363]
cosine [0.49743772 0.52998943 0.54679433 0.55351777]

Name ssr1_innorm1
euclid [0.49819577 0.53126409 0.54715244 0.55375193]
cosine [0.49822083 0.53135697 0.54721315 0.55378506]

Name norm1_innorm1
euclid [0.49743563 0.53003286 0.54682734 0.55346492]
cosine [0.49743772 0.52998943 0.54679433 0.55351777]

Name ssr1_norm1_innorm1
euclid [0.49819577 0.53126409 0.54715275 0.55375205]
cosine [0.49822083 0.53135697 0.54721315 0.55378506]

Name ssr2
euclid [0.4594785

KeyboardInterrupt: ignored